In [1]:
import skrub

import sempipes

dataset = skrub.datasets.fetch_toxicity()
records = skrub.var("toxicity_text", dataset.X.head(n=100))

records

2025-11-29 19:38:47,797 - INFO - SEMPIPES> Interactive mode detected, enabling code generation in preview mode.


<Var 'toxicity_text'>
Result:
―――――――
                                                 text
0   Elon Musk is a piece of shit, greedy capitalis...
1   The senile credit card shrill from Delaware ne...
2   He does that a lot -- makes everyone look good...
3                                          F*ck Lizzo
4   Epstein and trump were best buds!!! Pedophiles...
..                                                ...
95  How bout plain old Stupid Fuck\n Are the stupi...
96                        Demon Rats hate freedom !!!
97  If you squint, it looks like a fat guy bending...
98  Allen west is a war criminal and Obama killing...
99                               More libturd lunacy!

[100 rows x 1 columns]

In [2]:
records_with_extracted_features = records.sem_extract_features(
    nl_prompt="Extract various text features to help with categorizing the statement.",
    input_columns=["text"],
    name="text_features",
    output_columns={
        "sentiment_label": "Classify the overall sentiment of the tweet as 'positive', 'negative', or 'neutral.",
        "maybe_toxic": "Respond with '1' if a tweet is toxic and '0' if its text is not toxic.",
        "sweards_words": "A comma separated list of all the swear words in the text",
    },
    generate_via_code=True,
)

2025-11-29 19:38:48,117 - INFO - SEMPIPES> sempipes.sem_extract_features('['text']', '{'sentiment_label': "Classify the overall sentiment of the tweet as 'positive', 'negative', or 'neutral.", 'maybe_toxic': "Respond with '1' if a tweet is toxic and '0' if its text is not toxic.", 'sweards_words': 'A comma separated list of all the swear words in the text'}', 'Extract various text features to help with categorizing the statement.')
2025-11-29 19:38:48,118 - INFO - SEMPIPES> Generated possible columns: [{'feature_name': 'sentiment_label', 'feature_prompt': "Classify the overall sentiment of the tweet as 'positive', 'negative', or 'neutral.", 'input_columns': ['text']}, {'feature_name': 'maybe_toxic', 'feature_prompt': "Respond with '1' if a tweet is toxic and '0' if its text is not toxic.", 'input_columns': ['text']}, {'feature_name': 'sweards_words', 'feature_prompt': 'A comma separated list of all the swear words in the text', 'input_columns': ['text']}]
2025-11-29 19:38:48,119 - INFO

Chosen GPU: 3


Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use cuda:3
Device set to use cuda:3
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please

Chosen GPU: 2


Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use cuda:2
Device set to use cuda:2
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
2025-11-29 19:39:11,496 - INFO - SEMPIPES> Generated columns: ['text', 'sentiment_label', 'may

In [3]:
records_with_extracted_features

<Call 'extract_result'>
Result:
―――――――
                                                 text sentiment_label  \
0   Elon Musk is a piece of shit, greedy capitalis...        negative   
1   The senile credit card shrill from Delaware ne...        negative   
2   He does that a lot -- makes everyone look good...        negative   
3                                          F*ck Lizzo        negative   
4   Epstein and trump were best buds!!! Pedophiles...        negative   
..                                                ...             ...   
95  How bout plain old Stupid Fuck\n Are the stupi...        negative   
96                        Demon Rats hate freedom !!!        negative   
97  If you squint, it looks like a fat guy bending...        negative   
98  Allen west is a war criminal and Obama killing...        negative   
99                               More libturd lunacy!        negative   

    maybe_toxic sweards_words  
0             1          shit  
1             0                
2             1         bitch  
3             1                
4             1                
..          ...           ...  
95            1          fuck  
96            1                
97            1                
98            0                
99            0                

[100 rows x 4 columns]

In [4]:
sempipes.inspect_generated_code(records_with_extracted_features, "text_features")

import torch
import pandas as pd
from transformers import (
    pipeline,
    AutoTokenizer,
    AutoModelForSequenceClassification,
    AutoModelForTokenClassification,
)

def pick_gpu_by_free_mem_torch():
    assert torch.cuda.is_available(), "No CUDA device"
    free = []
    for i in range(torch.cuda.device_count()):
        # returns (free, total) in bytes for that device
        f, t = torch.cuda.mem_get_info(i)
        free.append((f, i))
    free.sort(reverse=True)
    _, idx = free[0]
    print(f"Chosen GPU: {idx}")
    return idx

gpu_idx = pick_gpu_by_free_mem_torch()
device = torch.device(f"cuda:{gpu_idx}") # Use the selected GPU for model inference

features_to_extract = [
    {
        'feature_name': 'sentiment_label',
        'feature_prompt': "Classify the overall sentiment of the tweet as 'positive', 'negative', or 'neutral.",
        'input_columns': ['text']
    },
    {
        'feature_name': 'maybe_toxic',
        'feature_prompt': "Respond with '1' if a tweet is toxic and '0' if its text is not toxic.",
        'input_columns': ['text']
    },
    {
        'feature_name': 'sweards_words',
        'feature_prompt': 'A comma separated list of all the swear words in the text',
        'input_columns': ['text']
    }
]

def extract_features(df: pd.DataFrame, features_to_extract: list[dict[str, object]]) -> pd.DataFrame:
    # Sentiment analysis pipeline (zero-shot, robust)
    sentiment_pipe = pipeline(
        "sentiment-analysis",
        model="cardiffnlp/twitter-roberta-base-sentiment-latest",
        device=device.index
    )

    # Toxicity detection pipeline (zero-shot, robust)
    toxic_pipe = pipeline(
        "text-classification",
        model="unitary/toxic-bert",
        device=device.index
    )

    # Swear words extraction: use a simple list of common swear words for demonstration
    # In production, consider using a more comprehensive list or a token classification model
    SWEAR_WORDS = set([
        "fuck", "shit", "bitch", "asshole", "bastard", "damn", "crap", "dick", "piss", "cock", "pussy", "slut", "douche", "bollocks", "bugger", "bloody", "bollocks", "arse", "wanker", "prick", "twat", "cunt"
    ])

    def extract_sentiment(text):
        try:
            result = sentiment_pipe(text, truncation=True)[0]
            label = result['label'].lower()
            if label == "positive":
                return "positive"
            elif label == "negative":
                return "negative"
            else:
                return "neutral"
        except Exception as e:
            return "neutral"

    def extract_toxic(text):
        try:
            result = toxic_pipe(text, truncation=True)[0]
            # The model returns 'toxic' or 'not toxic'
            if result['label'].lower() == "toxic" and result['score'] > 0.5:
                return 1
            else:
                return 0
        except Exception as e:
            return 0

    def extract_swear_words(text):
        import re
        # Lowercase and tokenize
        tokens = re.findall(r"\b\w+\b", text.lower())
        found = [word for word in tokens if word in SWEAR_WORDS]
        return ", ".join(sorted(set(found)))

    # Apply feature extraction
    if 'sentiment_label' in [f['feature_name'] for f in features_to_extract]:
        df['sentiment_label'] = df['text'].astype(str).apply(extract_sentiment)

    if 'maybe_toxic' in [f['feature_name'] for f in features_to_extract]:
        df['maybe_toxic'] = df['text'].astype(str).apply(extract_toxic)

    if 'sweards_words' in [f['feature_name'] for f in features_to_extract]:
        df['sweards_words'] = df['text'].astype(str).apply(extract_swear_words)

    return df